In [1]:
# Day 1
# Part one
with open('day1.txt') as fid:
    data = fid.read().strip().split('\n')
number_set = set('0123456789')
number_sum = 0
for d in data:
    numbers = [s for s in d if s in number_set]
    number_sum += int(numbers[0] + numbers[-1])
print(number_sum)

# Part two
number_list = [c for c in '0123456789'] + ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']
number_sum = 0
for d in data:
    first_val = min([(s, index) for index, n in enumerate(number_list) if (s := d.find(n))  != -1])[1] % 10
    last_val  = max([(s, index) for index, n in enumerate(number_list) if (s := d.rfind(n)) != -1])[1] % 10
    number_sum += int(str(first_val) + str(last_val))
print(number_sum)

54597
54504


In [2]:
# Day 2
# Part one
with open('day2.txt') as fid:
    data = fid.read().strip().split('\n')
games = []
colour_index = {'red':0, 'green':1, 'blue':2}
for line in data:
    turns = line.split(':')[1].split(';')
    this_game = []
    for t in turns:
        this_turn = [0, 0, 0]
        for colour_and_number in t.split(','):
            number, colour = colour_and_number.strip().split()
            this_turn[colour_index[colour]] = int(number)
        this_game.append(this_turn)
    games.append(this_game)
id_sum = 0
# 12 red cubes, 13 green cubes, and 14 blue cubes
limit = [12, 13, 14]
for game_id_minus_one, game in enumerate(games):
    game_impossible = False
    for t in game:
        if t[0] > limit[0] or t[1] > limit[1] or t[2] > limit[2]:
            game_impossible = True
            break
    if not game_impossible:
        id_sum += game_id_minus_one + 1
            
print(id_sum)

#Part two
power_sum = 0
for game in games:
    required = [0, 0, 0]
    for t in game:
        for index in range(3):
            required[index] = max(required[index], t[index])
    power_sum += required[0]*required[1]*required[2]
print(power_sum)

2679
77607


In [3]:
# Day 3
# Part one
from collections import defaultdict
with open('day3.txt') as fid:
    engine_map = [c for c in fid.read()]

Nx = engine_map.index('\n') + 1
Ny = len(engine_map)//Nx

all_numbers = []
dict_of_potential_gears = defaultdict(list)
number = []
digits = set(c for c in '0123456789')
digits_and_dot = {'.', '\n'} | digits
for flatidx, c in enumerate(engine_map):
    if c in digits:
        number.append(c)
    elif number:
        #New number, check:
        j, i = flatidx // Nx, flatidx % Nx
        ndigits = len(number)
        check_point_offsets = [(-1,0), (ndigits,0)] + [(nd,1) for nd in range(-1, ndigits + 1)] + [(nd,-1) for nd in range(-1, ndigits + 1)]
        start_point = (j, i - ndigits)
        add_number = False
        for di, dj in check_point_offsets:
            jj, ii = start_point[0] + dj, start_point[1] + di
            if not ((0 <= ii < Nx) and (0 <= jj < Ny)):
                continue
            flat_idx_test = jj*Nx + ii
            if engine_map[flat_idx_test] not in digits_and_dot:
                add_number = True
                if engine_map[flat_idx_test] == '*':
                    dict_of_potential_gears[flat_idx_test].append(int(''.join(number)))
                # We cannot break here, since there could be multiple * around a number
        if add_number:
            all_numbers.append(int(''.join(number)))
        number = []

print(sum(all_numbers))
# Part two
gear_ratio_sum = 0
for key, val in dict_of_potential_gears.items():
    if len(val) != 2:
        continue
    gear_ratio_sum += val[0]*val[1]
print(gear_ratio_sum)

537832
81939900


In [62]:
# Day 4
# Part one
with open('day4.txt') as fid:
    data = fid.read().strip().split('\n')

scratch_wins = []
for line in data:
    index_colon, index_sep = line.find(':'), line.find('|')
    winning_numbers = set(int(d) for d in line[(index_colon + 1):index_sep].split())
    played_numbers  = set(int(d) for d in line[(index_sep + 1):].split())
    scratch_wins.append(len(winning_numbers & played_numbers))
print(sum(2**(s - 1) for s in scratch_wins if s > 0))

# Part two
card_stash = [1]*len(scratch_wins)
for i, wins in enumerate(scratch_wins):
    for j in range(wins):
        if (i + 1 + j) >= len(scratch_wins):
            break
        card_stash[i + 1 + j] += card_stash[i]
print(sum(card_stash))

24160
5659035


In [60]:
# Day 5
# Part one
with open('day5.txt') as fid:
    data = fid.read().strip().split('\n\n')

seeds = [int(num) for num in data[0].split(':')[1].split()]
if_list_dict = {}
for j in range(1, len(data)):
    tmp = data[j].split('\n')
    if_list_dict[tmp[0][:-1]] = [[int(c) for c in s.split()] for s in tmp[1:]]

def apply_if_from_list(x, if_list):
    for il in if_list:
        if il[1] <= x < (il[1] + il[2]):
            x += il[0] - il[1]
            break
    return x

location_numbers = []
for s in seeds:
    for val in if_list_dict.values():
        s = apply_if_from_list(s, val)
    location_numbers.append(s)
print(min(location_numbers))

#Part two
def apply_if_from_list_inverse(x, if_list):
    for il in if_list:
        if il[0] <= x < (il[0] + il[2]):
            x += il[1] - il[0]
            break
    return x

test_location = 0
not_found = True
while not_found:
    s = test_location
    for val in reversed(if_list_dict.values()):
        s = apply_if_from_list_inverse(s, val)
    # Test if seed is in range:
    for i in range(0, len(seeds), 2):
        if seeds[i] <= s < (seeds[i] + seeds[i + 1]):
            not_found = False
            print(test_location)
            break
    test_location += 1

535088217
51399228


In [108]:
# Day 6
# Part one
import math
with open('day6.txt') as fid:
    data = fid.read().strip().split('\n')
times = [int(d) for d in (data[0].split(':')[1]).split()]
dists = [int(d) for d in (data[1].split(':')[1]).split()]
def get_margin(T, R):
    sqrt_arg = (T/2)**2 - R - 1
    if sqrt_arg < 0:
        return 0
    tmin = np.ceil(T/2 - np.sqrt(sqrt_arg))
    tmax = np.floor(T/2 + np.sqrt(sqrt_arg))
    return int(tmax - tmin + 1)

print(math.prod(m for T, R in zip(times, dists) if (m := get_margin(T, R))))

# Part two
time = int(''.join([str(d) for d in times]))
dist = int(''.join([str(d) for d in dists]))
print(get_margin(time, dist))

5133600
40651271


In [165]:
# Day 7
# Part one
import math
with open('day7.txt') as fid:
    data = fid.read().strip().split('\n')

def get_hand_type(hand):
    cards, count = np.unique(list(hand), return_counts=True)
    hand_type = max(count)
    if hand_type == 3 and len(count) == 2:
        hand_type = 3.5
    elif hand_type == 2 and len(count) == 3:
        hand_type = 2.5
    return hand_type

card_values = {c:(n + 2) for n, c in enumerate('23456789TJQKA')}
hand_values = [(get_hand_type(handbid[0]),) + tuple(card_values[c] for c in handbid[0]) + (int(handbid[1]),)
              for d in data if (handbid := d.split())]
print(sum((rankm1 + 1)*h[6] for rankm1, h in enumerate(sorted(hand_values))))

# Part two
card_values = {c:(n + 1) for n, c in enumerate('J23456789TQKA')}
def get_hand_type_with_jokers(hand):
    njokers = sum(1 for c in hand if c == 'J')
    if njokers == 5:
        return 5
    cards, count = np.unique(list(hand.replace('J', '')), return_counts=True)
    c = cards[np.argmax(count)]
    hand = hand.replace('J', c)
    return get_hand_type(hand)

hand_values = [(get_hand_type_with_jokers(handbid[0]),) + tuple(card_values[c] for c in handbid[0]) + (int(handbid[1]),)
              for d in data if (handbid := d.split())]
print(sum((rankm1 + 1)*h[6] for rankm1, h in enumerate(sorted(hand_values))))

250602641
251037509


In [272]:
# Day 8
# Part one
with open('day8.txt') as fid:
    directions, network = fid.read().strip().split('\n\n')

lts = ''.join(map(chr, range(ord('A'), ord('Z') + 1)))
Nlts = len(lts)
node_to_index = {(c1 + c2 + c3):index1*Nlts**2 + index2*Nlts + index3 
                 for index1, c1 in enumerate(lts) 
                 for index2, c2 in enumerate(lts)
                 for index3, c3 in enumerate(lts)}

directions_indices = [0 if c == 'L' else 1 for c in directions]
lts_set = set(lts)
node_list = [(-1,-1)]*Nlts**3
for line in network.strip().split('\n'):
    tmp = ''.join(c for c in line if c in lts_set)
    node, left, rght = [node_to_index[tmp[3*j:3*(j + 1)]] for j in range(3)]
    node_list[node] = (left, rght)

pos = 0
steps = 0
while pos != (Nlts**3 - 1):
    dir_index = directions_indices[steps % len(directions_indices)]
    pos = node_list[pos][dir_index]
    steps += 1
print(steps)

# Part two
import itertools
ghost_pos = [index for index in [idx1*Nlts**2 + idx2*Nlts for idx1 in range(Nlts) for idx2 in range(Nlts)] 
             if node_list[index] != (-1, -1)]

def get_steps_and_pos_to_next_final(pos, start_dir_index):
    steps = 0
    while steps == 0 or pos%Nlts != (Nlts - 1):
        dir_index = directions_indices[(start_dir_index + steps) % len(directions_indices)]
        pos = node_list[pos][dir_index]
        steps += 1
    return steps, pos

only_steps = [get_steps_and_pos_to_next_final(pos, 0)[0] for pos in ghost_pos]

def prime_factorisation(n):
    for i in itertools.chain([2], itertools.count(3, 2)):
        if n <= 1:
            break
        while n % i == 0:
            n //= i
            yield i
            
prime_factors = set()
for steps in only_steps:
    prime_factors |= set(prime_factorisation(steps))
print(math.prod(prime_factors))

14257
16187743689077
